Пишем код
---

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import sqlalchemy as db
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table, Float, Boolean, Text, MetaData
from sqlalchemy.orm import sessionmaker, relationship, declarative_base, join
from google.colab import drive


drive.mount('/content/drive')

db_file_path = '/content/drive/MyDrive/GTDb.db'

engine = create_engine(f'sqlite:///{db_file_path}')
Base = declarative_base()

Session = sessionmaker(bind=engine)
session = Session()

Mounted at /content/drive


In [ ]:
def fig_histogram_nkill_alltime():
    fig_histogram = px.histogram(ter_alltime, x='iyear', y='nkill', title='Гистограмма количества жертв по годам',
                              labels={'iyear': 'Год', 'nkill': 'Количество жертв'},
                              histfunc='sum', nbins=len(year_victims['iyear'].unique())+1)   # 1993 года нет в базе
    fig_histogram.show()


def fig_line_country_nkill():
    fig = px.line(year_victims, x='iyear', y='nkill', color='country_txt',
                  title='Количество жертв по годам для каждой страны',
                  labels={'iyear': 'Год', 'nkill': 'Количество жертв', 'country_txt': 'Страна'})

    fig.update_layout(xaxis_title='Год',
                      yaxis_title='Количество жертв',
                      legend_title='Страна')

    fig.show()


def top_nkill_country_compare_periods():
    print(' ')
    print('Количество смертей за промежутки времени и их прирост в % (страны).')
    print(' ')
    period_2001_2011 = year_victims[(year_victims['iyear'] >= 2001) & (year_victims['iyear'] <= 2011)]
    period_2012_2017 = year_victims[(year_victims['iyear'] >= 2012) & (year_victims['iyear'] <= 2017)]

    total_2001_2011 = period_2001_2011.groupby('country_txt', as_index=False)['nkill'].sum()
    total_2012_2017 = period_2012_2017.groupby('country_txt', as_index=False)['nkill'].sum()

    total_2001_2011.rename(columns={'nkill': 'nkill_2001_2011'}, inplace=True)
    total_2012_2017.rename(columns={'nkill': 'nkill_2012_2017'}, inplace=True)

    result = pd.merge(total_2001_2011, total_2012_2017, on='country_txt', how='outer')

    result['growth'] = round(((result['nkill_2012_2017']/result['nkill_2001_2011'])-1)*100, 2)

    result['growth2'] = round(result['nkill_2012_2017'] - result['nkill_2001_2011'], 0)

    result = result[result['nkill_2012_2017'] > 5000]

    result = result.sort_values(by='nkill_2012_2017', ascending=False)

    table = result[['country_txt', 'nkill_2001_2011', 'nkill_2012_2017', 'growth', 'growth2']]

    table.rename(columns={'country_txt': 'Страна',
                        'nkill_2001_2011': '2001-2011',
                        'nkill_2012_2017': '2012-2017',
                        'growth': 'Рост в %',
                        'growth2': 'Рост в чел.'
                        }, inplace=True)

    return table


def fig_line_weapsubtype_nkill():
    fig = px.line(year_guntype, x='iyear', y='nkill', color='weapsubtype1_txt',
                title='Количество жертв по годам для каждой страны',
                labels={'iyear': 'Год', 'nkill': 'Количество жертв', 'weapsubtype1_txt': 'Страна'})

    fig.update_layout(xaxis_title='Год',
                    yaxis_title='Количество жертв',
                    legend_title='Страна')

    fig.show()

def top_nkill_weapsubtype_compare_periods():
    print('Количество смертей за промежутки времени и их прирост в % (тип оружия).')
    print(' ')
    period_1970_2000 = year_guntype[(year_guntype['iyear'] >= 1970) & (year_guntype['iyear'] <= 2000)]
    period_2001_2011 = year_guntype[(year_guntype['iyear'] >= 2001) & (year_guntype['iyear'] <= 2011)]
    period_2012_2017 = year_guntype[(year_guntype['iyear'] >= 2012) & (year_guntype['iyear'] <= 2017)]

    total_1970_2000 = period_1970_2000.groupby('weapsubtype1_txt', as_index=False)['nkill'].sum()
    total_2001_2011 = period_2001_2011.groupby('weapsubtype1_txt', as_index=False)['nkill'].sum()
    total_2012_2017 = period_2012_2017.groupby('weapsubtype1_txt', as_index=False)['nkill'].sum()

    total_1970_2000.rename(columns={'nkill': 'nkill_1970_2000'}, inplace=True)
    total_2001_2011.rename(columns={'nkill': 'nkill_2001_2011'}, inplace=True)
    total_2012_2017.rename(columns={'nkill': 'nkill_2012_2017'}, inplace=True)

    merged_2000_2011 = pd.merge(total_1970_2000, total_2001_2011, on='weapsubtype1_txt', how='outer')
    result = pd.merge(merged_2000_2011, total_2012_2017, on='weapsubtype1_txt', how='outer')

    result = result[result['nkill_2012_2017'] > 2000]

    result = result.sort_values(by='nkill_2012_2017', ascending=False)

    table = result[['weapsubtype1_txt','nkill_1970_2000', 'nkill_2001_2011', 'nkill_2012_2017']]

    table.rename(columns={'weapsubtype1_txt': 'Тип оружия',
                        'nkill_1970_2000': '1970-2000',
                        'nkill_2001_2011': '2001-2011',
                        'nkill_2012_2017': '2012-2017'
                        }, inplace=True)

    return table

def fig_line_weapsubtype_nkill():
    fig = px.line(year_guntype, x='iyear', y='nkill', color='weapsubtype1_txt',
                title='Количество жертв по годам для каждой страны',
                labels={'iyear': 'Год', 'nkill': 'Количество жертв', 'weapsubtype1_txt': 'Страна'})

    fig.update_layout(xaxis_title='Год',
                    yaxis_title='Количество жертв',
                    legend_title='Страна')

    fig.show()

In [ ]:
class Region(Base):
    __tablename__ = 'region'

    id = Column(Integer, primary_key=True)
    region_txt = Column(String, nullable=False)

    countries = relationship("Country", back_populates="region")


class Country(Base):
    __tablename__ = 'country'

    id = Column(Integer, primary_key=True)
    country_txt = Column(String)
    region_id = Column(Integer, ForeignKey('region.id'))

    region = relationship("Region", back_populates="countries")
    cities = relationship("City", back_populates="country")


class City(Base):
    __tablename__ = 'city'

    id = Column(Integer, primary_key=True)
    city = Column(String)
    country_id = Column(Integer, ForeignKey('country.id'))

    country = relationship("Country", back_populates="cities")
    incidents = relationship("Incident", back_populates="city")


class TypeAttack(Base):
    __tablename__ = 'type_attack'

    id = Column(Integer, primary_key=True)
    attacktype1_txt = Column(String)

    incidents = relationship("Incident", back_populates="type_attack")


class Corp(Base):
    __tablename__ = 'corp'

    id = Column(Integer, primary_key=True)
    corp1 = Column(String)

    incidents = relationship("Incident", back_populates="corp")


class Natlty(Base):
    __tablename__ = 'natlty'

    id = Column(Integer, primary_key=True)
    natlty1_txt = Column(String)

    incidents = relationship("Incident", back_populates="natlty")


class WeaponType(Base):
    __tablename__ = 'weapon_type'

    id = Column(Integer, primary_key=True)
    weaptype1_txt = Column(String)

    weapon_sub_types = relationship("WeaponSubType", back_populates="weapon_type")


class WeaponSubType(Base):
    __tablename__ = 'weapon_sub_type'

    id = Column(Integer, primary_key=True)
    weapsubtype1_txt = Column(String)
    weapon_type_id = Column(Integer, ForeignKey('weapon_type.id'))

    weapon_type = relationship("WeaponType", back_populates="weapon_sub_types")
    incidents = relationship("Incident", back_populates="weapon_sub_type")


class Propextent(Base):
    __tablename__ = 'propextent'

    id = Column(Integer, primary_key=True)
    propextent_txt = Column(String)

    incidents = relationship("Incident", back_populates="propextent")


class Claimmode(Base):
    __tablename__ = 'claimmode'

    id = Column(Integer, primary_key=True)
    claimmode_txt = Column(String)

    incidents = relationship("Incident", back_populates="claimmode")


class Target(Base):
    __tablename__ = 'target'

    id = Column(Integer, primary_key=True)
    target1 = Column(Text)

    incidents = relationship("Incident", back_populates="target")


class TargetType(Base):
    __tablename__ = 'target_type'

    id = Column(Integer, primary_key=True)
    targtype1_txt = Column(String)

    target_sub_types = relationship("TargetSubType", back_populates="target_type")


class TargetSubType(Base):
    __tablename__ = 'target_sub_type'

    id = Column(Integer, primary_key=True)
    targsubtype1_txt = Column(String)
    target_type_id = Column(Integer, ForeignKey('target_type.id'))

    target_type = relationship("TargetType", back_populates="target_sub_types")
    incidents = relationship("Incident", back_populates="target_sub_type")


class TypeIncident(Base):
    __tablename__ = 'type_incident'

    id = Column(Integer, primary_key=True)
    alternative_txt = Column(String)

    incidents = relationship("Incident", back_populates="type_incident")


class Incident(Base):
    __tablename__ = 'incident'

    id = Column(Integer, primary_key=True)
    iyear = Column(Integer)
    imonth = Column(Integer)
    iday = Column(Integer)
    city_id = Column(Integer, ForeignKey('city.id'))
    type_incident_id = Column(Integer, ForeignKey('type_incident.id'))
    type_attack_id = Column(Integer, ForeignKey('type_attack.id'))
    target_sub_type_id = Column(Integer, ForeignKey('target_sub_type.id'))
    corp_id = Column(Integer, ForeignKey('corp.id'))
    target_id = Column(Integer, ForeignKey('target.id'))
    natlty_id = Column(Integer, ForeignKey('natlty.id'))
    claimmode_id = Column(Integer, ForeignKey('claimmode.id'))
    weapon_sub_type_id = Column(Integer, ForeignKey('weapon_sub_type.id'))
    propextent_id = Column(Integer, ForeignKey('propextent.id'))

    city = relationship("City", back_populates="incidents")
    type_incident = relationship("TypeIncident", back_populates="incidents")
    type_attack = relationship("TypeAttack", back_populates="incidents")
    target_sub_type = relationship("TargetSubType", back_populates="incidents")
    corp = relationship("Corp", back_populates="incidents")
    target = relationship("Target", back_populates="incidents")
    natlty = relationship("Natlty", back_populates="incidents")
    claimmode = relationship("Claimmode", back_populates="incidents")
    weapon_sub_type = relationship("WeaponSubType", back_populates="incidents")
    propextent = relationship("Propextent", back_populates="incidents")
    other_data = relationship("OtherData", back_populates="incident", uselist = False)
    international = relationship("InternationalData", back_populates="incident", uselist=False)
    place_spec = relationship("PlaceSpec", back_populates="incident", uselist=False)
    hostages = relationship("Hostages", back_populates="incident", uselist=False)
    attack_spec = relationship("AttackSpec", back_populates="incident", uselist=False)
    victims = relationship("Victims", back_populates="incident", uselist=False)
    properties = relationship("Properties", back_populates="incident", uselist=False)


class OtherData(Base):
    __tablename__ = 'other_data'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    addnotes = Column(Text)
    scite1 = Column(Text)
    scite2 = Column(Text)
    scite3 = Column(Text)

    incident = relationship("Incident", back_populates="other_data", uselist=False)


class InternationalData(Base):
    __tablename__ = 'international_data'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    INT_LOG = Column(Integer)
    INT_IDEO = Column(Integer)
    INT_MISC = Column(Integer)
    INT_ANY = Column(Integer)

    incident = relationship("Incident", back_populates="international", uselist=False)


class PlaceSpec(Base):
    __tablename__ = 'placespec'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    latitude = Column(Float)
    longitude = Column(Float)
    specificity = Column(Integer)
    vicinity = Column(Integer)
    location = Column(Text)
    summary = Column(Text)
    motive = Column(Text)

    incident = relationship("Incident", back_populates="place_spec", uselist=False)


class Hostages(Base):
    __tablename__ = 'hostages'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    ishostkid = Column(Integer)
    nhostkid = Column(Integer)
    nhostkidus = Column(Integer)
    ndays = Column(Integer)
    ransom = Column(Integer)
    ransomamt = Column(Integer)
    ransompaid = Column(Integer)
    hostkidoutcome_txt = Column(String)

    incident = relationship("Incident", back_populates="hostages", uselist=False)


class AttackSpec(Base):
    __tablename__ = 'attackspec'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    crit1 = Column(Boolean)
    crit2 = Column(Boolean)
    crit3 = Column(Boolean)
    doubtterr = Column(Integer)
    multiple = Column(Boolean)
    success = Column(Boolean)
    suicide = Column(Boolean)

    incident = relationship("Incident", back_populates="attack_spec", uselist=False)


class Victims(Base):
    __tablename__ = 'victims'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    nkill = Column(Integer)
    nkillus = Column(Integer)
    nkilter = Column(Integer)
    nwound = Column(Integer)
    nwoundus = Column(Integer)
    nwoundte = Column(Integer)

    incident = relationship("Incident", back_populates="victims", uselist=False)

class Properties(Base):
    __tablename__ = 'properties'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    properties = Column(Integer)
    propvalue = Column(Integer)
    propcomment = Column(Text)

    incident = relationship("Incident", back_populates="properties", uselist=False)


In [ ]:
with Session() as session:
    query = db.select(
        Victims.nkill, Incident.id, Incident.iyear, Incident.imonth, Incident.iday, Region.region_txt,
        Country.country_txt, City.city, Corp.corp1, PlaceSpec.summary
        ).outerjoin(City, City.id == Incident.city_id
        ).outerjoin(PlaceSpec, PlaceSpec.id == Incident.id
        ).outerjoin(Corp, Corp.id == Incident.corp_id
        ).outerjoin(Country, Country.id == City.country_id
        ).outerjoin(Region, Region.id == Country.region_id
        ).outerjoin(Victims, Victims.id == Incident.id
        ).where(Country.country_txt == 'Russia').where(Victims.nkill > 50)
    result = session.execute(query)
    for row in enumerate(result):
        print(row)

(0, (121, 59561, 1995, 6, 14, 'Eastern Europe', 'Russia', 'Budyonnovsk', None, None))
(1, (64, 68677, 1999, 3, 19, 'Eastern Europe', 'Russia', 'Vladikavkaz', 'Civilian', "03/19/1999: A bomb was detonated in the central market of Vladikavkaz, North Ossetia, Russia. The explosion killed 64 people and injured 104. The equ ... (249 characters truncated) ... he Jordanian Chechnya field commander 'Hattab' were thought to be the masterminds. Two men, aged 18 and 33 were sought in connection with the attack."))
(2, (64, 69394, 1999, 9, 4, 'Eastern Europe', 'Russia', 'Buynaksk', 'Russian Army', '09/04/1999: In Buynaksk, Dagestan, Russia, sixty-four people were killed and over 100 were wounded when a GAZ-52 vehicle containing one ton of explos ... (455 characters truncated) ... ere released under an amnesty law after receiving three-year sentences. In 2006, Magomed Salikhov, charged with organizing the attack, was acquitted.'))
(3, (90, 69411, 1999, 9, 9, 'Eastern Europe', 'Russia', 'Pechatniki'

In [ ]:
query = db.select(Incident.iyear, Region.region_txt, Country.country_txt,
                  City.city, TypeAttack.attacktype1_txt, TypeIncident.alternative_txt,
                  TargetType.targtype1_txt, WeaponSubType.weapsubtype1_txt, Victims.nkill
                  ).outerjoin(City, City.id == Incident.city_id
                  ).outerjoin(Country, Country.id == City.country_id
                  ).outerjoin(Region, Region.id == Country.region_id
                  ).outerjoin(TypeAttack, TypeAttack.id == Incident.type_attack_id
                  ).outerjoin(TypeIncident, TypeIncident.id == Incident.type_incident_id
                  ).outerjoin(TargetSubType, TargetSubType.id == Incident.target_sub_type_id
                  ).outerjoin(TargetType, TargetType.id == TargetSubType.target_type_id
                  ).outerjoin(WeaponSubType, WeaponSubType.id == Incident.weapon_sub_type_id
                  ).outerjoin(WeaponType, WeaponType.id == WeaponSubType.weapon_type_id
                  ).outerjoin(Victims, Victims.id == Incident.id)

In [ ]:
ter_alltime = pd.read_sql(query, engine)

In [ ]:
year_victims = ter_alltime[['country_txt', 'iyear', 'nkill']]

In [ ]:
year_victims.dropna()

,country_txt,iyear,nkill
0,Colombia,1970,1.0
1,Mexico,1970,0.0
2,Burkina Faso,1970,1.0
5,Egypt,1970,0.0
6,Uruguay,1970,0.0
...,...,...,...
181686,Somalia,2017,1.0
181687,Syria,2017,2.0
181688,Philippines,2017,0.0
181689,India,2017,0.0


In [ ]:
year_victims = year_victims.groupby(['iyear', 'country_txt'], as_index=False)['nkill'].sum()

In [ ]:
year_victims_12_17 = year_victims.where(year_victims.iyear > 2011)

In [ ]:
year_guntype = ter_alltime[['weapsubtype1_txt', 'iyear', 'nkill']]

In [ ]:
year_guntype = year_guntype.groupby(['iyear', 'weapsubtype1_txt'], as_index=False)['nkill'].sum()

---
Анализ
---

In [ ]:
fig_histogram_nkill_alltime()

---
Малое количество жертв до 1980 года относительно других лет можно объяснить недостатком данных за этот период.

Наблюдается сильный рост жертв терроризма начиная с 2012 года. Проанализируем подробнее и выведем статистику по странам:

In [ ]:
fig_line_country_nkill()

---
Можно заметить, что в некоторых странах рост количества смертей в результате террористических актов резко вырос. Возьмем топ стран по количеству смертей за 2012-2017 гг.

In [ ]:
table = top_nkill_country_compare_periods()
table

 
$Количество смертей за промежутки времени и их прирост в % (страны).
 


,Страна,2001-2011,2012-2017,Рост в %,Рост в чел.
68,Iraq,12699.0,36783.0,189.65,24084.0
0,Afghanistan,7339.0,29450.0,301.28,22111.0
107,Nigeria,1583.0,19542.0,1134.49,17959.0
41,Egypt,11462.0,12159.0,6.08,697.0
109,Pakistan,8097.0,11856.0,46.42,3759.0
140,Syria,105.0,11530.0,10880.95,11425.0
130,Somalia,2358.0,8020.0,240.12,5662.0
161,Yemen,836.0,7039.0,741.99,6203.0
21,Burkina Faso,2910.0,6948.0,138.76,4038.0


Все указанные страны в наибольшей степени повлияли на статистику по следующим причинам:
1. Окончание Арабской весны и активизация группировок. Начиная с 2010 года по странам арабского мира прокатился каскад революций в результате которых произошли некоторые радикальные преобразования: активизация различных террористических группировок, а именно ИГ (Ирак, Сирия), Боко-Харам (Нигерия), Джихадистское восстание группировок в Буркина-Фасо, Хуситы (Йемен).
2. Количество группировок в Афганистане начало резко расти с начала 2010-х, также как и количество террактов против мирного населения ввиду многолетней иностранной интервенции США (с 2001 года) во время операции "Несокрушимая свобода".
3. Продолжение серии террактов в Египте и Пакистане от различных джихадистских групировок.
4. После серии поражений от правительственных войск в Сомали в 2014 году, группировка Аш-Шабаб начала серию террактов против мирного населения, унесшей жизни около 7-8 тыс. человек.

Итого: "избыточная" смертность от террористических актов в 2012-2017гг. по сравнению с прошлыми годами вызвана событиями в арабских странах, в большей части ввиду Арабской весны.

---
Теперь перейдём к анализу использования различных типов оружия для осуществления террора также по годам.

In [ ]:
fig_line_weapsubtype_nkill()

---
Также посмотрим на количество жертв от наиболее распространённых типов оружия по нескольким временным промежуткам.

In [ ]:
table = top_nkill_weapsubtype_compare_periods()
table

Количество смертей за промежутки времени и их прирост в % (тип оружия).
 


,Тип оружия,1970-2000,2001-2011,2012-2017
27,Unknown Gun Type,15570.0,20816.0,50311.0
29,Vehicle,6300.0,18179.0,32905.0
18,"Projectile (rockets, mortars, RPGs, etc.)",3068.0,2780.0,18008.0
26,Unknown Explosive Type,13161.0,6448.0,17567.0
24,Suicide (carried bodily by human being),634.0,11149.0,16886.0
1,Automatic or Semi-Automatic Rifle,68314.0,3976.0,5885.0
10,Landmine,2919.0,2112.0,4561.0
6,Grenade,1842.0,1323.0,2640.0


---
Как видно из данных выше, примерно до 1995 года в мире подавляющее большинство актов совершалось c использованием ***огнестрельного оружия***, используемого в различных боевых столкновениях. Однако ближе к
нашему времени, акцент сместился на использование ***взрывчатых веществ*** (в частности, акты с использованием *заминированных транспортных средств*, *террористов-смертников*), а также более широкое применение ***снарядов взрывного действия*** во время вооруженных конфликтов. К этому привело удешевление производства взрывчатых веществ, развитие технологий вооружения, увеличение их точности, а так же эффективность в поражении большого количества целей без прямого столкновения.